In [1]:
import numpy as np
import pandas as pd

#### Опрежедение функций удалениея и расчета таргета

In [2]:
def drop(df : pd.DataFrame):
    df.drop('date', axis=1, inplace=True)
    df.drop('geo_lat', axis=1, inplace=True)
    df.drop('geo_lon', axis=1, inplace=True)
    df.drop('building_type', axis=1, inplace=True)
    df.drop('level', axis=1, inplace=True)
    df.drop('levels', axis=1, inplace=True)
    df.drop('kitchen_area', axis=1, inplace=True)
    df.drop('distance_from_centre', axis=1, inplace=True)
    df.drop('Unnamed: 0.1', axis=1, inplace=True)
    df.drop('Unnamed: 0', axis=1, inplace=True)
    df.drop('price', axis=1, inplace=True)
    df.drop('time', axis=1, inplace=True)
    df.drop('area', axis=1, inplace=True)
    df.drop('timestamp', axis=1, inplace=True)

def drop_segment(df : pd.DataFrame):
    df.drop('rooms', axis=1, inplace=True)
    df.drop('object_type', axis=1, inplace=True)
    df.drop('district', axis=1, inplace=True)

def price_metr(df : pd.DataFrame):
    df['target'] = df['price'] / df['area']

def AVG_mouth(df, start_end_year, start_end_month ):
        start_year = start_end_year[0]
        end_year = start_end_year[1]
        df_category = pd.DataFrame( columns = ['timestamp', 'target' , 'size'])
        for year in range(start_year,end_year+1):
            if year == start_year:
                start_month = start_end_month[0]
            else:
                start_month = 1
            if year == end_year:
                end_month = start_end_month[1]
            else:
                end_month = 12
            for month in range(start_month,end_month+1):
                # Вырезаем данный месяц из исходного датасета
                df_category_add = df.loc[(df['month'] == month) & (df['year'] == year)].copy()
                # Удаляем уже не нежные столбцы
                df_category_add.drop('year', axis=1, inplace=True)
                df_category_add.drop('month', axis=1, inplace=True)
                # обрезаем выбромы (хвостики) из данного месяцы
                df_category_add = df_category_add[(df_category_add['target'] > df_category_add['target'].quantile(0.25)) & (df_category_add['target'] < df_category_add['target'].quantile(0.9))]
                df_category = pd.concat([df_category,pd.DataFrame({'timestamp' : [f"{year}-{month}-01"] , 'target' : [df_category_add['target'].median()] , 'size': [df_category_add.shape[0]] })])
            
        df_category.reset_index(drop = True, inplace = True)
        return df_category

#### Загрузка dataseta

In [3]:
df = pd.read_csv('../data/processed/msk_2018_2021.csv')

In [4]:
df = df[df['rooms'] < 4]

#### Удаление не нужных столбцов

In [5]:
price_metr(df)
drop(df)

In [6]:
print(df.shape)
df.head(10)

(369054, 6)


,rooms,object_type,target,year,month,district
0,1,11,148400.000000,2018,3,поселение Московский
1,2,1,177777.777778,2018,3,район Вешняки
2,3,1,177195.685670,2018,4,Черкизово
3,2,1,193181.818182,2018,4,Нагорный район
4,1,1,273809.523810,2018,4,район Раменки
5,3,1,261250.000000,2018,5,район Замоскворечье
6,3,1,184899.845917,2018,5,район Чертаново Северное
7,1,1,179687.500000,2018,6,район Кузьминки
8,3,1,142583.436341,2018,6,Рязанский район
9,2,1,158863.636364,2018,7,Можайский


In [7]:
start_year = df['year'].min()
start_month = df.loc[df['year'] == start_year]['month'].min()
end_year = df['year'].max()
end_month = df.loc[df['year'] == end_year]['month'].max()

#### Находим уникальные значения в стольце district, object_type, rooms столбов

In [8]:
list_unique_district = df.district.unique()
list_unique_object_type = df.object_type.unique()
list_unique_rooms = df.rooms.unique()

In [9]:
print(list_unique_district.shape)
print(list_unique_object_type)
print(list_unique_rooms)
print(list_unique_district.shape[0] * list_unique_object_type.shape[0] * list_unique_rooms.shape[0])

(146,)
[11  1]
[1 2 3 0]
1168


In [10]:
# grouped = df.groupby(['rooms','object_type','district','year','month']) #['target'].mean()
# grouped.agg({'target': 'mean'})
# print(grouped['rooms'].head())
#grouped[grouped['year'] == 2020 & grouped['district'] == 'Академический район' & grouped['object_type'] == 1 & grouped['rooms'] == 1]

In [11]:
list_df_segment = []
start_end_year = start_year , end_year
start_end_month = start_month, end_month
for dis in list_unique_district:
    for obj in list_unique_object_type:
        for room in list_unique_rooms:
            df_segment = df.copy()
            drop_segment(df_segment)
            list_df_segment.append(AVG_mouth(df_segment, start_end_year, start_end_month))           

In [12]:
df_category = list_df_segment[718]

In [14]:
df_category.tail(20)

,timestamp,target,size
19,2019-10-01,192187.500000,6804
20,2019-11-01,187500.000000,8268
21,2019-12-01,188888.888889,5860
22,2020-1-01,192000.000000,4810
23,2020-2-01,197058.823529,5613
24,2020-3-01,193557.022648,7344
25,2020-4-01,198575.653495,4158
26,2020-5-01,200000.000000,5066
27,2020-6-01,204545.454545,8018
28,2020-7-01,208595.124403,11223
